In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Running on {device}")

Running on cuda


In [3]:
import whisper
model = whisper.load_model("large", download_root="models").to(device=device) # load "large" model

In [4]:
import os
import pandas as pd
import time
import json

In [6]:
folder_path = "mp3_files"

if os.path.exists(folder_path):
    files = os.listdir(folder_path) # get list of files
    mp3_files = [file for file in files if file.lower().endswith(".mp3")] # get all files endswith '.mp3'
    for mp3 in mp3_files:
        if mp3 not in done:
            full_path = os.path.abspath(os.path.join(folder_path, mp3))
            print(f"Transcribing {mp3}")

            transcribe = model.transcribe(f"{full_path}", language="tr")
        
            print("Writing...")
            df = pd.DataFrame({"name": mp3, "transcript": [transcribe["text"]]})

            df.to_csv('transcript_files/transcriptions.csv', sep='|', mode="a", index=False)

Transcribing Zaman Makinesi 1973 _ Gürgen Öz, Seda Bakan.mp3
Writing...
Transcribing Zıkkımın Kökü _ Türk Filmi 1992.mp3
Writing...
Transcribing Çakal   türk filmleri 2013 tek parça 1.mp3
Writing...
Transcribing Çöpçüler Kralı _ FULL HD.mp3
Writing...
Transcribing Ölümlü Dünya _ Feyyaz Yiğit FULL HD Komedi Filmi İzle.mp3
Writing...
Transcribing Öz Hakiki Karakol _  Türk Filmi Tek Parça (HD).mp3
Writing...
Transcribing Öğretmen _ Kemal Sunal Komedi Filmi.mp3
Writing...
Transcribing Üç Kağıtçı Türk Filmi _ FULL HD _ Kemal Sunal Filmleri.mp3
Writing...
Transcribing İki Kafadar _ Türk Komedi Filmi Tek Parça (HD).mp3
Writing...
Transcribing İkimize Bir Dünya (Yerli Film) HD.mp3
Writing...
Transcribing İncir Çekirdeği _ Türk Filmi Dram Filmi _ Full Film İzle.mp3
Writing...
Transcribing İçimdeki Ses _ Türk Komedi Filmi (Tek Parça).mp3
Writing...
Transcribing Şark Bülbülü Türk Filmi _ RESTORASYONLU _ Kemal Sunal Filmleri.mp3
Writing...
Transcribing Şerr_i Cin Adlı Yerli Korku Filmi (2023).mp3


In [7]:
d = pd.read_csv('transcript_files/transcriptions.csv', sep='|')

,name,transcript
0,A.R.O.G _ Cem Yılmaz Türk Komedi Filmi.mp3,İzlediğiniz için teşekkür ederim. Altyazı M.K...
1,name,transcript
2,Abimm _ Türk Filmi Tek Parça (HD).mp3,Bu dizinin betimlemesi TRT tarafından Sesli B...
3,name,transcript
4,Acıların Kadını Bergen _ Yerli Film _ Weco Fil...,Yıllar yıllık dert yolunda Ne ilk neden sonun...


In [8]:
d.shape

(383, 2)

In [12]:
d[d["name"] != "name"].shape